> 最近买了房，买房的时候认识了个很可爱的女孩，心动之下想追她，她是做房屋中介的，由于我们毕竟不同行业，我想了解她更多，能为她提供一些方便（说不定啊，这是我自认为），所以就想找个房屋信息网站，爬爬实用的信息并做一下有价值的数据分析。于是，先从房天下租房信息开始下手。

# 房天下租房信息

## 待完成

### 地图api信息经纬度


### 遍历户型页

### 遍历楼盘页

### excel输出

区域，小区，地址，租金，户型，出租方式，特色，朝向，面积，楼层，装修，房源亮点，周边配套，交通出行，配套设施

### sqlite3保存

### 机器学习分析
 

# 楼盘页信息

以香江天赋页为例，开发思路如下：
1. 从具体楼盘页开始获取信息
2. 从具体楼盘页获取详情页和户型页
3. 获取详情页信息
4. 获取户型页信息
5. 打印excel
6. 保存sqlite3
7. 进行机器学习算法流程

## 导入模块

In [2]:
import requests
import re
from pyquery import PyQuery as pq
import random

## 定位楼盘首页

1. 预设首页

2. 设置随机User-Agent，进行低级防反爬操作

In [3]:
url = 'http://xiangjiangtianfu.fang.com/'
# url_detail='http://xiangjiangtianfu.fang.com/house/2811174364/housedetail.htm'
# url_huxing='https://xiangjiangtianfu.fang.com/photo/list_900_2811174364.htm'

In [4]:
user_agent_list = [
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
"Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
"Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
]

headers = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Host':'xiangjiangtianfu.fang.com',
    'Referer':'http://gz.newhouse.fang.com/house/s/',
    'Upgrade-Insecure-Requests':'1',
    'User-Agent': random.choice(user_agent_list)
}

## 网页解码

房天下的网页直接get的话，会发现源码中文部分无法解析，从网上查询到可以用**gb18030**解码，那就无需过多费事了

In [7]:
rq = requests.get(url,headers=headers)
doc=pq(rq.content.decode('gb18030'))

### 其他解码方法
```python
#gzip解压缩
import gzip
import urllib
req = urllib.request.Request(url=url,headers=headers,method='GET')
content_ori = urllib.request.urlopen(req).read()
content = gzip.decompress(content_ori).decode("gb18030")

#转换为latin-1,二进制形式
content=rq.text.encode('latin-1').decode("gb18030")
#有没有发现是多此一举，没错直接使用content属性获取的即为二进制，无需在编码为latin-1了，此处仅供参考
```

### 导航栏信息

为了方便的通过首页即可定位其他信息页，我们直接从导航栏下手。获取其中另外两个关键信息页。

1. 定位导航栏
2. 获取楼盘详情页
3. 获取户型页

### 页面信息定位
#### pyquery进行selector定位

便捷的方式即通过chrome等等浏览器的开发者模式在**Elements**页，对节点右键copy Selector，然后通过PyQuery方法实现定位。
```python 
from pyquery import PyQuery as pq
#pq解析html源码
doc=pq(html)
#寻找selector
nav=doc('#orginalNaviBox a')
```
#### etree进行xpath定位

```python

from lxml import etree
et=etree.HTML(content)
et.xpath('/html/body/div[3]/div[3]/div[2]/div[1]/div[2]/div[2]/a[1]/div/p[1]/text()')

```


In [6]:
#首页导航
nav=doc('#orginalNaviBox a')
url_detail='https:'+nav.eq(1).attr('href')
url_huxing='https:'+nav.eq(2).attr('href')
print(url_detail)
print(url_huxing)

https://xiangjiangtianfu.fang.com/house/2811174364/housedetail.htm
https://xiangjiangtianfu.fang.com/house/2811174364/dongtai.htm


### 首页信息打印
1. 楼盘名称
2. 别名
3. 均价
4. 价格说明
5. 地图api信息获取经纬度


In [9]:
house=doc('body > div.main_1200.tf > div.firstbox > div.firstright.fr > div.information > div.inf_left1 > div > h1 > strong').text()
alias=doc('body > div.main_1200.tf > div.firstbox > div.firstright.fr > div.information > div.inf_left1 > div > span').text()
price=doc('body > div.main_1200.tf > div.firstbox > div.firstright.fr > div.information > div:nth-child(2) > div.inf_left.fl > span').text()
price_detail=doc('body > div.main_1200.tf > div.firstbox > div.firstright.fr > div.information > div:nth-child(2) > div.inf_right.fl.pr > a.pra01.fl > div > p:nth-child(2)').text()
print(house)
print(alias)
print(price)
print(price_detail)


香江天赋
别名：香江翡翠馨园
23900
香江天赋在售88-127㎡复式和平层产品，平层单位均价23000元/平，复式单位均价24800元/平，带装修出售。


## 楼盘详情页

In [10]:
req_d = requests.get(url_detail,headers=headers)
doc_d = pq(req_d.content.decode("gb18030"))

In [11]:
#全部信息
print(doc_d(".main-item").eq(0).text())

基本信息
价格： 均价约23900元/平方米
用户点评： 4.46 [32条点评]
物业类别：
普通住宅
项目特色：
品牌地产
建筑类别：
塔楼 高层
装修状况：
非毛坯
产权年限：
普通住宅:70年
环线位置：
暂无资料
开发 商：
增城香江房地产有限公司
楼盘地址：
广园快速路开创大道
销售信息
销售状态：
在售
楼盘优惠：
暂无
开盘时间：
2018年12月30日6栋开盘[开盘时间详情]
交房时间：
2020年12月6#交房
售楼地址：
广州广园快速路开创大道北香江天赋
咨询电话：
400-890-0000 转 885462
主力户型：
三居(2) 四居(6) 五居(12)
预售许可证：
预售许可证
发证时间
绑定楼栋
20181174
2018-12-27
6#
周边设施
交通地铁： 13号线-新塘。 项目交通路网三横两纵，通达性强。紧邻广园、广深、107国道三干道，距离天河区约24公里。拥开创大道、源章大道两条南北主线，约8公里可达萝岗科学城及新塘中心。距离TOD交通枢纽仅8公里，拥有3条国家高铁（广深高铁、广汕高铁和京九高铁）、2条城际轨道（穗莞深和新白广2条城际轨道）、3条地铁路线（13号、16号和23号）以及有轨电车交汇，珠江新城、东莞、深圳、香港等都距离改项目不远（约38—100公里），形成珠三角半小时生活圈。 公交车两站路直接接驳13号线沙村站，交通核心优势得天独厚，享受极致便利。 项目设有3条楼巴路线，分别是到天河宏发大厦、黄埔文化宫及萝岗科学城皇冠假日酒店，楼巴每天运行，出行无忧。目前项目有4条公交车线路，分别是437（项目开往开发东区，接驳南岗地铁站）、572（项目开往开发西区）、358（项目开往萝岗中心）、新塘6路（项目开往塘中心，接驳沙村地铁站）。
幼儿园翡翠绿洲幼儿园、东方绿洲幼儿园。
中小学1所小学（翡翠绿洲香江小学）。2所初高中（香江小学初中部、香江中学初中部及高中部）等，香江集团联手省一级育才实验学校创办香江中学，由育才中学直管，实现育才系三校联动，是增城至高升学率学校。其中香江中学国际部重金聘请了美国金牌校长Jeffrey Maher坐镇，开设华南首家AP顶石课程，而2018年的AP班毕业生更是创下78%被美国综合排名前50大学录取的记录。
大学项目周边现已落成或在建近20余所优质学校，项目5公里内：广州第二中学选址于项目周边

### 详情页基本信息

In [12]:
#基本信息
basic_info=doc_d(".main-item").eq(0)
price=basic_info(".main-info-price")('em').text()
dianping_href=basic_info(".main-info-comment")('a').attr('href')
dianping=basic_info(".main-info-comment")('span').eq(2).text()
list_clearfix=basic_info("ul li") #basic_info(".list.clearfix")
wuyeleibie=list_clearfix.eq(0)(".list-right").text()
xiangmutese=list_clearfix.eq(1)(".list-right")(".tag").text()
jianzhuleibie=list_clearfix.eq(2)(".list-right").text()
zhuangxiu=list_clearfix.eq(3)(".list-right").text()
chanquan=list_clearfix.eq(4)(".list-right").text()
huanxianweizhi=list_clearfix.eq(5)(".list-right").text()
kaifashang=list_clearfix.eq(6)(".list-right-text").text()
loupandizhi=list_clearfix.eq(7)(".list-right-text").text()
print(price)
print(dianping_href)
print(dianping)
print(wuyeleibie)
print(xiangmutese)
print(jianzhuleibie)
print(zhuangxiu)
print(chanquan)
print(huanxianweizhi)
print(kaifashang)
print(loupandizhi)



均价约23900元/平方米
//xiangjiangtianfu.fang.com/dianping/
4.46
普通住宅
品牌地产
塔楼 高层
非毛坯
普通住宅:70年
暂无资料
增城香江房地产有限公司
广园快速路开创大道


### 详情页销售信息

In [13]:
#销售信息
sale_info=doc_d(".main-item").eq(1)
list_clearfix=sale_info(".list.clearfix")('li')
sale_status=list_clearfix.eq(0)(".list-right").text()
youhui=list_clearfix.eq(1)(".list-right").text()
kaipanshijian=list_clearfix.eq(2)(".list-right").text()
jiaofangdate=list_clearfix.eq(3)(".list-right").text()
salesaddr=list_clearfix.eq(4)(".list-right").text()
zixun=list_clearfix.eq(5)(".list-right").text()
zhulihuxing=list_clearfix.eq(6)(".list-right-text").text()

print(sale_status)
print(youhui)
print(kaipanshijian)
print(jiaofangdate)
print(salesaddr)
print(zixun)
print(zhulihuxing)


在售
暂无
2018年12月30日6栋开盘[开盘时间详情]
2020年12月6#交房
广州广园快速路开创大道北香江天赋
400-890-0000 转 885462
三居(2) 四居(6) 五居(12)


### 详情页周边设施

In [14]:
#周边设施
surroundingfacility=doc_d(".main-item").eq(2)
list_clearfix=surroundingfacility(".sheshi_zb")('li')
traffic=list_clearfix.eq(0).text()
kindergarten=list_clearfix.eq(1).text()
middleschool=list_clearfix.eq(2).text()
univercity=list_clearfix.eq(3).text()
coodepartment=list_clearfix.eq(4).text()
hospital=list_clearfix.eq(5).text()
bank=list_clearfix.eq(6).text()
rest=list_clearfix.eq(7).text()
facilities=list_clearfix.eq(8).text()
print(traffic)
print(kindergarten)
print(middleschool)
print(univercity)
print(coodepartment)
print(hospital)
print(bank)
print(rest)
print(facilities)


交通地铁： 13号线-新塘。 项目交通路网三横两纵，通达性强。紧邻广园、广深、107国道三干道，距离天河区约24公里。拥开创大道、源章大道两条南北主线，约8公里可达萝岗科学城及新塘中心。距离TOD交通枢纽仅8公里，拥有3条国家高铁（广深高铁、广汕高铁和京九高铁）、2条城际轨道（穗莞深和新白广2条城际轨道）、3条地铁路线（13号、16号和23号）以及有轨电车交汇，珠江新城、东莞、深圳、香港等都距离改项目不远（约38—100公里），形成珠三角半小时生活圈。 公交车两站路直接接驳13号线沙村站，交通核心优势得天独厚，享受极致便利。 项目设有3条楼巴路线，分别是到天河宏发大厦、黄埔文化宫及萝岗科学城皇冠假日酒店，楼巴每天运行，出行无忧。目前项目有4条公交车线路，分别是437（项目开往开发东区，接驳南岗地铁站）、572（项目开往开发西区）、358（项目开往萝岗中心）、新塘6路（项目开往塘中心，接驳沙村地铁站）。
幼儿园翡翠绿洲幼儿园、东方绿洲幼儿园。
中小学1所小学（翡翠绿洲香江小学）。2所初高中（香江小学初中部、香江中学初中部及高中部）等，香江集团联手省一级育才实验学校创办香江中学，由育才中学直管，实现育才系三校联动，是增城至高升学率学校。其中香江中学国际部重金聘请了美国金牌校长Jeffrey Maher坐镇，开设华南首家AP顶石课程，而2018年的AP班毕业生更是创下78%被美国综合排名前50大学录取的记录。
大学项目周边现已落成或在建近20余所优质学校，项目5公里内：广州第二中学选址于项目周边，合生育才学校（500米步行可达的），广州二中系增城区应元学校（距离项目2.5公里），英国四大公学之一的费德斯（距离项目2.7公里），项目5-10公里内：美国格兰纳达山特许学校，修仕倍励实验学校，广州南方中英文学校等学校，形成了10公里的教育长廊，名校塔尖教育荟萃林立。
综合商场克拉U街、克拉街市、乐家生鲜。3大城市级超级商圈，珠江新城商圈、天河城商圈、萝岗万达商圈距离项目平均约225公里。
医院暨华医院（3.8公里）、广东省水电二局医院（8.6公里）、新塘医院（10公里）、岭南医院-中山大学附属第三医院（14公里）、南方医科大学南方医院增城院区（10公里）。
银行建设银行
其他周边景观：香江天赋保留原生自然环境，建筑与自然共生，被陈家林原始森林环抱，壮阔的天然森林，藏风聚气，山林

### 详情页价格信息

In [15]:
#价格信息,这里没什么好看的
price=doc_d(".main-item").eq(4)
print(price)

<div class="main-item">
            	<h3>
                   <a href="//xiangjiangtianfu.fang.com/house/2811174364/fangjia.htm" target="_blank" class="price-trend">查看价格走势 &gt;</a>
                	价格信息
                </h3>
                <div class="main-table">
                    <div class="table-part">
                       <table cellpadding="0" cellspacing="0">
                          <tr>
                              <td width="110">记录时间</td>
                              <td width="90">均价</td>
                              <td width="90">起价</td>
                              <td width="342">价格描述</td>
                          </tr>
						  															 <tr>
						        <td>2019-01-06</td>
						        <td>约23900元/㎡</td>
						        <td>约23000元/㎡</td>
						        <td class="td-left">香江天赋在售88-127㎡复式和平层产品，平层单位均价23000元/平，复式单位均价24800元/平，带装修出售。</td>
								</tr>
													  															 <tr>
						        <td>2018-12-30</td>
						        <td>约23900元/㎡</td>
			

## 户型页

### 获取ajax返回json数据

In [16]:
req_h = requests.get(url_huxing,headers=headers)
doc_h = pq(req_h.content.decode("gb18030"))

In [23]:
huxing_list=doc_h('.xc_list').text()
query_string={
'newcode': 2812199376,
'count': 'false',
'start': 12,
'limit': 12,
'room': 'all',
'city': '%E5%B9%BF%E5%B7%9E'
}
url_house=url+"house/ajaxrequest/householdlist_get.php?"
print(url_house)
headers = {
    'Referer':'https://xiangjiangtianfu.fang.com/photo/list_900_2811174364.htm',
    'User-Agent': random.choice(user_agent_list),
    'x-requested-with': 'XMLHttpRequest'
}
houses_ajax=requests.get(url_house,params=query_string,headers=headers)


http://xiangjiangtianfu.fang.com/house/ajaxrequest/householdlist_get.php?


In [28]:
houses_ajax.json()

[{'reference_price': '152.19',
  'reference_price_type': '万元/套',
  'IntervalPrice': '0-0',
  'IntervalPriceType': '万元/套',
  'picID': '202016249',
  'hx_score': '4.80',
  'fz_score': '5.00',
  'cg_score': '5.00',
  'bj_score': '5.00',
  'cc_score': '4.00',
  'df_score': '5.00',
  'hx_desp': '各个空间户型方正，方便室内家具布置；全明通透户型，居住舒适度较高，整个空间采光充足，利于后期居住；整体空间布局合理，能够保证动静分离和居室通风，方便使用；卫生间偏小，通过合理选择卫浴家具也可让卫生间实现其使用功能。',
  'huxing_feature': '客厅朝南,卧室朝南,厨卫不对门',
  'room': '2',
  'hall': '2',
  'toilet': '1',
  'kitchen': '1',
  'buildingarea': '80.10',
  'huxing_saling': '1',
  'houseimageurl': '//img11.soufunimg.com/viewimage/house/2018_12/31/M02/11/6D/ChCE4Vwp6LGIOdhRAATl3yOdrhYABIxpAL_BjAABOX3903/220x150.png',
  'housetitle': 'A1栋建面80平03户型',
  'pic_sort': '50',
  'updatedate': '2019-01-04 22:06:21',
  'livingarea': '65.17',
  'id': '0',
  'open_type': '',
  'price_interface_show': 'y',
  'status': '在售',
  'style': '',
  'typearea': '建面',
  'hxtNum': '',
  'onsaleNum': {'OnsaleNum': ''}},
 {'reference_price':

### 遍历所有户型页

## 参考文档

https://www.jianshu.com/p/639a67bc75b9

地图定位api:https://ditu.fang.com/?c=channel&a=ajaxXiaoquMapSearch&x1=113.557556&y1=23.1417&distance=2&strNewCode=2811174364&city=gz